### 콘텐츠 기반 필터링 실습 -tmdb 5000 영화 데이터세트

In [41]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies=pd.read_csv('tmdb_5000_movies.csv')
movies.shape
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [42]:
movies_df=movies[['id','title','genres','vote_average','vote_count','popularity',
                 'keywords','overview']]

In [43]:
pd.set_option('max_colwidth',100)
movies_df[['genres','keywords']][:1]
#여러개로 합쳐져 있음 

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [44]:
from ast import literal_eval
movies_df['genres']=movies_df['genres'].apply(literal_eval)
movies_df['keywords']=movies_df['keywords'].apply(literal_eval)


In [45]:
movies_df['genres']=movies_df['genres'].apply(lambda x:[y['name'] for y in x])
movies_df['keywords']=movies_df['keywords'].apply(lambda x:[y['name'] for y in x])
movies_df[['genres','keywords']][:1]
#name 에 해당하는 값만 추출 

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


* 장르 콘텐츠 유사도 측정
- genres 안에 여러 장르 있음 -> 문자열 변경-> COUNT 피처 벡터화 -> 코사인 유사도 ->유사도가 높은 영화 중 평점 높은 순 추천

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

#공백문자로 word단위가 구분되는 문자열로 변환
movies_df['genres_literal']= movies_df['genres'].apply(lambda x: (' ').join(x))
count_vect=CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat=count_vect.fit_transform(movies_df['genres_literal']) #별도 컬럼으로 저장 
genre_mat.shape

(4803, 276)

In [47]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim=cosine_similarity(genre_mat, genre_mat)
genre_sim.shape
genre_sim[:3]

#genre_sim  moviesdf , genreliteral 칼럼을 피처 백터화한 행렬 (genre_mat) 데이터행별 유사도 정보
#갖고 있으며 결국 movies_df 행별 장르 유사도도 갖고있음

array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ],
       [0.59628479, 1.        , 0.4       , ..., 0.        , 0.        ,
        0.        ],
       [0.4472136 , 0.4       , 1.        , ..., 0.        , 0.        ,
        0.        ]])

In [48]:
#유사도 높은 순으로 위치 인덱스 값 추출 
genre_sim_sorted_ind=genre_sim.argsort()[:,::-1]
genre_sim_sorted_ind
#첫줄 보면 0번 레코드의 경우, 자신제외 3494 가 유사도 제일 높고 2401 제일 낮음 

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   2, 1740, 1542, ..., 3000, 2999, 2401],
       ...,
       [4800, 3809, 1895, ..., 2229, 2230,    0],
       [4802, 1594, 1596, ..., 3204, 3205,    0],
       [4802, 4710, 4521, ..., 3140, 3141,    0]], dtype=int64)

* 장르 콘텐츠 필터링을 이용한 영화추천

In [49]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie=df[df['title']== title_name]
    title_index=title_movie.index.values
    similar_indexes=sorted_ind[title_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes=similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [50]:
#대부와 비슷한 영화 추천
silmilar_movies= find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
silmilar_movies[['title','vote_average']]
# 1등 대부 2 , Mi America 평점 0임 -> 개선

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [51]:
#적은 관객이 특정 영화에 만점 > 왜곡  vote count 적은게 더 점수가높음 
movies_df[['title','vote_average','vote_count']].sort_values('vote_average',ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


* IMDB 평가횟수 가중치 부여된 평점 이용  

In [55]:
C=movies_df['vote_average'].mean()
m=movies_df['vote_count'].quantile(0.6)

#m : 투표횟수에 다른 가중치 조절 , 높이면 평점 투표횟수가 많은 가중 평점 부여 

def weighted_voted_average(record):
    v= record['vote_count']
    R= record['vote_average']
    return( (v/(v+m) * R ) + ((m/m+v))*C)

movies_df['weighted_vote']=movies.apply(weighted_voted_average, axis=1)

In [57]:
#새롭게 부여된 weighted vote 평점 순서 상위 10개 영화 추출 

movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote',ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
96,Inception,8.1,83793.523123,13752
65,The Dark Knight,8.2,73132.289868,12002
0,Avatar,7.2,71900.697559,11800
16,The Avengers,7.4,71754.678913,11776
788,Deadpool,7.4,66996.677427,10995
95,Interstellar,8.1,66217.553660,10867
287,Django Unchained,7.8,61538.456936,10099
94,Guardians of the Galaxy,7.9,59363.638291,9742
426,The Hunger Games,6.9,57614.214284,9455
127,Mad Max: Fury Road,7.2,57443.921401,9427


* 장르 유사성이 높은 영화를 top_n의 두배수 만큼 선정뒤 weighted_vote 칼럼이 높은 순으로 top_n 추출

In [59]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie=df[df['title']== title_name]
    title_index=title_movie.index.values
    
    #top_n 2배에 해당하는 장르 유사성이 높은 인덱스 추출 
    similar_indexes=sorted_ind[title_index, :(top_n*2)]
    similar_indexes=similar_indexes.reshape(-1)
    #기준영화 인덱스 제외
    similar_indexes=similar_indexes[similar_indexes != title_index]
    
    return df.iloc[similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n]

#한번 더 대부와 유사도 높은 영화 추천 
silmilar_movies= find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
silmilar_movies[['title','vote_average','weighted_vote']]

,title,vote_average,weighted_vote
883,Catch Me If You Can,7.7,23132.898869
2731,The Godfather: Part II,8.3,20349.232225
1847,GoodFellas,8.2,19069.737037
1149,American Hustle,6.8,17112.825419
3866,City of God,8.1,11064.018512
281,American Gangster,7.4,9162.470613
1663,Once Upon a Time in America,8.2,6524.714313
588,Wall Street: Money Never Sleeps,5.8,3012.845308
2839,Rounders,6.9,2684.298813
4041,This Is England,7.4,2221.214114


- 이전보단 훨씬 나아진 영화 추천 목록 하지만 개인이 좋아하는 성향을 반영하기엔 부족 (콘텐츠 기반 필터링) 
-좋아하는 배우나 감독을 보고 영화를 선택하는 경우 많음 